# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [22]:
grouped_data = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
grouped_data = data.reset_index()
grouped_data.head()

,index,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [23]:
pivot = pd.pivot_table(grouped_data,values='Quantity',index='ProductName',columns='CustomerID',fill_value=0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [24]:
distances = squareform(pdist(pivot.T, 'euclidean'))
print(distances)

[[  0.          10.72380529   9.64365076 ... 197.94948851 218.8241303
  203.79892051]
 [ 10.72380529   0.          10.67707825 ... 197.34994299 218.62525014
  203.95342606]
 [  9.64365076  10.67707825   0.         ... 197.32460566 218.02981447
  203.80628057]
 ...
 [197.94948851 197.34994299 197.32460566 ...   0.         266.9269563
  266.9269563 ]
 [218.8241303  218.62525014 218.02981447 ... 266.9269563    0.
  269.25824036]
 [203.79892051 203.95342606 203.80628057 ... 266.9269563  269.25824036
    0.        ]]


In [25]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [26]:
most_similaries = pd.DataFrame(distances.loc[33].sort_values(ascending=False)[1:6])
most_similaries

,33
CustomerID,
3909,0.095358
264,0.093953
3531,0.093953
2503,0.093498
3305,0.093051


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [29]:
most_similaries_arr = [3909,264,3531,2503,3305]
most_similaries2 = grouped_data.loc[grouped_data['CustomerID'].isin(most_similaries_arr)]
most_similaries2

,index,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
130,130,3909,Keri,Stafford,509197,270,Cheese Cloth No 100,1
159,159,264,Virgil,Small,24688,270,Cheese Cloth No 100,1
448,448,3909,Keri,Stafford,1834282,432,Beer - Original Organic Lager,1
887,887,3305,Bernard,O'Neill,2025094,102,"Rosemary - Primerba, Paste",1
928,928,3909,Keri,Stafford,4776765,417,"Lamb - Pieces, Diced",1
...,...,...,...,...,...,...,...,...
66085,66085,3909,Keri,Stafford,3584400,296,Wine - Valpolicella Masi,1
66436,66436,264,Virgil,Small,3526383,378,Bread Foccacia Whole,1
66703,66703,3909,Keri,Stafford,739301,71,Cheese - Wine,1
67528,67528,3305,Bernard,O'Neill,4034755,412,Lettuce - California Mix,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [40]:
grouped_data2 = most_similaries2.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False)
grouped_data2.head()

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [63]:
merge_data = grouped_data2.merge(pivot, on='ProductName')
merge_data = merge_data[merge_data[33]==0].sort_values('Quantity',ascending=False)[0:5]
print(merge_data.index.tolist())
merge_data

['Chocolate - Dark', 'Bay Leaf', 'Oranges - Navel, 72', 'Wine - Ej Gallo Sierra Valley', 'Pecan Raisin - Tarts']


,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Chocolate - Dark,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
Bay Leaf,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,25,25,0,0,0
"Oranges - Navel, 72",3,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Wine - Ej Gallo Sierra Valley,3,0,0,0,1,0,0,0,0,0,...,0,0,25,25,0,25,0,0,0,0
Pecan Raisin - Tarts,3,0,0,1,0,0,0,0,0,0,...,0,25,0,0,25,0,25,0,0,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [64]:
# No sé ni por dónde se meterle mano a esto!!!

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.